# Modelo de programación lineal Süer et. al.

In [ ]:
import pandas as pd
import gurobipy as grb

In [ ]:
def main():
    
    #Entradas:Colección de trabajos A[0…ith], mediante una tabla con el formato "JOB,PT,Deadline" en la primera fila donde la columna JOB provee el indicador del trabajo, PT el tiempo de procesamiento y Deadline 
    #         plazo maximo de entrega del trabajo i, tal que 0≤i≤ith.
    #Salidas:Máxima cantidad de trabajos de A[0…ith] que se pueden elaborar sin violar su tiempo límite de entrega.
    
    datafile=input("Ingrese el nombre del archivo donde posee las solicitudes a agendar acompañado del tipo de archivo (.csv o .txt)\n")
    data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

    data["JOB"]="J"+data["JOB"].map(str)
    dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

    #Los trabajos son ordenados de acuerdo con la regla EDD (Plazo de entrega más temprano), tal que Deadline0≤…Deadlinei…≤DeadlineI,
    #JOB0≤…JOBi…≤JOBI y son añadidos dentro de un diccionario, tomando como clave su indice JOBi y valor la tupla (JOBi,Deadlinei,PTi).    
    
    dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
    ordDict={}
    cont=0

    for i in dictJobs:
        ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
        cont+=1
    
    p = []
    d = []
    for i in ordDict.keys():
        p.append(ordDict[i][2])
        d.append(ordDict[i][1])

    #Conjuntos:
    #Se crea el conjunto N y M, tal que N={0,..,i,..,ith} y M={0,..,j,..,jth} donde ith hace referencia al ultimo trabajo del 
    #conjunto y jth a la ultima máquina del conjunto.        
    jth=2
    ith=len(ordDict)
    N = range(ith)
    M = range(jth)
    indices=[(i,j) for i in N for j in M]

    #Inicializar modelo
    mdl = grb.Model("modelo")

    #Variables:
    #x: 1 si el trabajo i esta a tiempo en la maquina j; 0 de lo contrario.
    x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')

    #Funcion objetivo: Maximizar el numero de trabajos a tiempo.    
    mdl.setObjective(sum(x[(i,j)] for i in N for j in M))

    #Restricciones:
    #R1: Asegura que cada trabajo sea desempeñado unicamente en una maquina.    
    mdl.addConstrs((sum(x[(i,j)] for j in M) <= 1 for i in N), name='A')
    
    #R2: Asegura que eltiempo de finalización de cada trabajo sea menor que su plazo de entrega.
    for k in N:
        for j in M:
            mdl.addConstr(sum(x[(i,j)]*p[i] for i in N if i<=k)<=d[k])

    mdl.ModelSense = grb.GRB.MAXIMIZE
    mdl.optimize()
    
main()